# Previsão de churn de clientes de uma empresa

## Descrição do problema

O objetivo desse projeto é utilizar informações sobre clientes de uma empresa para identificar os clientes mais propensos ao churn (cancelamento). Essa análise permite que a empresa direcione gerentes para tais clientes e não de forma aleatória. 

As informações sobre os clientes são as seguintes:
 
 - Nome 
 - Idade
 - Compra total
 - Gerente de conta (se possuí ou não possui)
 - Anos (tempo como cliente)
 - N. de sites que utilizam o serviço
 - Data que o último contato foi integrado ("Onboard" date)
 - Endereço do cliente
 - Nome da empresa do cliente
 - Churn (Sim ou Não)

## Carregamento dos dados e pré-processamento

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('churn').getOrCreate()

22/05/12 08:13:43 WARN Utils: Your hostname, gabriel-ana resolves to a loopback address: 127.0.1.1; using 192.168.0.14 instead (on interface wlp1s0)
22/05/12 08:13:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/12 08:13:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# importa os dados
data = spark.read.csv('customer_churn.csv', header = True, inferSchema=True)

In [4]:
data.head(4)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date='2013-08-13 00:38:46', Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1),
 Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date='2016-06-29 06:20:07', Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1),
 Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date='2014-04-22 12:43:12', Location='13120 Daniel Mount Angelabury, WY 30645-4695', Company='Smith Inc', Churn=1)]

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
# verifica se existem dados faltantes
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|    0|  0|             0|              0|    0|        0|           0|       0|      0|    0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

Obviamente o nome do cliente e sua empresa não interferem no churn. Nós consideramos ainda que 'Onboard_date' e 'Location' podem ser descartadas na análise. Portanto consideramos somente as variáveis 'Age', 'Total_Purchase','Years' e 'Num_Sites' na análise.

## Machine Learning

In [9]:
# agrupa as variáveis que serão utilizadas na classificação
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase','Years','Num_Sites']
                                       , outputCol='features')

In [10]:
from pyspark.ml.classification import LogisticRegression

In [11]:
from pyspark.ml import Pipeline

In [12]:
# modelo de regressão logística
log_reg_churn = LogisticRegression(labelCol='Churn')

In [13]:
# cria o pipeline com as estapas do pré-processamento
pipeline = Pipeline(stages=[assembler, log_reg_churn])

In [14]:
# divide os dados entre treino e teste
train_data, test_data = data.randomSplit([0.7, 0.3])

In [15]:
# ajusta o modelo nos dados de treino
fit_model = pipeline.fit(train_data)

22/05/12 08:14:07 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/05/12 08:14:07 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [16]:
# aplica o modelo nos dados de teste
results = fit_model.transform(test_data)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [31]:
# computa a área sob a curva ROC
AUC = my_eval.evaluate(results)

In [32]:
AUC

0.7534464475079533

Esse valor para a AUC é bem razoável e indica que o molelo é eficiente em identificar o churn.

## Aplicando o modelo para uma nova base de dados de clientes

In [38]:
# importa a nova base de dados
new_data = spark.read.csv('new_customers.csv', header = True, inferSchema=True)

In [40]:
# predições sobre os clientes
new_results = fit_model.transform(new_data)
new_results.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

